In [ ]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, MaxPool2D
import cv2



In [ ]:
df = pd.read_csv('data/crop_detection/Crop_details.csv')
df.head()

In [ ]:
categories = os.listdir('data/crop_detection/kag2')
categories

In [ ]:
getData = ImageDataGenerator(rescale=1/255,
                             shear_range=0.3,
                             horizontal_flip=True,
                             vertical_flip=True,
                             rotation_range=90)

In [ ]:
kag2_data = getData.flow_from_directory("data/crop_detection/kag2",
                                        target_size=(224,224),
                                        class_mode='categorical',
                                        shuffle=True,
                                        batch_size=32)

In [ ]:
vgg_model = VGG16(input_shape=(224,224,3),
                  include_top=False,
                  weights='imagenet',
                  pooling='avg')
for layers in vgg_model.layers:
    layers.trainable = False

In [ ]:
model = Sequential()
model.add(vgg_model)
model.add(Dense(256,activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(64,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(5,activation="softmax"))

In [ ]:
from keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.001),
              loss="categorical_crossentropy",
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
num_train_samples = 804
batch_size = 64
epochs = 50

final_model = model.fit(kag2_data,
                                  steps_per_epoch=num_train_samples/batch_size,
                                  epochs=epochs)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1/255)
crop_data = test_datagen.flow_from_directory('data/crop_detection/crop_images',
                                             target_size=(224,224),
                                             class_mode='categorical',
                                             shuffle=True,
                                             batch_size=32)

In [ ]:
plt.plot(final_model.history['loss'],label='train_loss')
plt.plot(final_model.history['accuracy'],label="train_acc")
plt.legend()
plt.show()

In [ ]:
class_labels = crop_data.class_indices
class_labels = {v: k for k, v in class_labels.items()}
classes = list(class_labels.values())
Y_pred = model.predict(crop_data)
y_pred = np.argmax(Y_pred, axis = 1)

In [ ]:
def predict_crop(path, actual, class_labels):
    img = cv2.imread(path)
    img = cv2.resize(img, (224,224))
    img = np.array(img).reshape((1,224,224,3))
    Y_pred = model.predict(img)
    y_pred = np.argmax(Y_pred, axis=1)
    if y_pred == actual:
        print("Correct Prediction")
    else:
        print("Wrong Prediction")
    print('Accual class "{0}" and predicted class "{1}"'.format(class_labels[int(y_pred)],class_labels[actual]))
    
    

In [ ]:
predict_crop('data/crop_detection/kag2/rice/rice024ahs.jpeg',2,class_labels)

In [ ]:
predict_crop('data/crop_detection/test_crop_image/sugarcane fields.jpg',3,class_labels)